In [5]:
!pip uninstall torch torchvision -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.4.1
Uninstalling torch-2.4.1:
  Successfully uninstalled torch-2.4.1
ERROR: Exception:
Traceback (most recent call last):
  File "/software/slurm/spackages/linux-rocky8-x86_64/gcc-12.2.0/anaconda3-2023.09-0-3mhml42fa64byxqyd5fig5tbih625dp2/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/software/slurm/spackages/linux-rocky8-x86_64/gcc-12.2.0/anaconda3-2023.09-0-3mhml42fa64byxqyd5fig5tbih625dp2/lib/python3.11/site-packages/pip/_internal/commands/uninstall.py", line 110, in run
    uninstall_pathset.commit()
  File "/software/slurm/spackages/linux-rocky8-x86_64/gcc-12.2.0/anaconda3-2023.09-0-3mhml42fa64byxqyd5fig5tbih625dp2/lib/python3.11/site-packages/pip/_internal/req/req_uninstall.py", line 432, in commit
    self._moved_paths.commit()
  File "/software/slurm/spackages/linux-rocky8-x86_64/gcc-12.2.0/anaconda3-2023.09-0-3mhml42fa64byx

In [24]:
import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("TorchVision version:", torchvision.__version__)


ModuleNotFoundError: No module named 'torch._C._dynamo.guards'; 'torch._C._dynamo' is not a package

In [28]:
!pip3 uninstall torch torchvision torchaudio -y
!pip3 cache purge


Found existing installation: torch 2.5.1+cu118
Uninstalling torch-2.5.1+cu118:
  Successfully uninstalled torch-2.5.1+cu118
Found existing installation: torchvision 0.20.1+cu118
Uninstalling torchvision-0.20.1+cu118:
  Successfully uninstalled torchvision-0.20.1+cu118
Found existing installation: torchaudio 2.5.1+cu118
Uninstalling torchaudio-2.5.1+cu118:
  Successfully uninstalled torchaudio-2.5.1+cu118
Files removed: 3


In [29]:
!pip install torch==2.5.1+cu118 torchvision==0.20.1+cu118 torchaudio --index-url https://download.pytorch.org/whl/cu118


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 93.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.6 MB/s eta 0:00:0000:01:00:01


In [3]:
import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("TorchVision version:", torchvision.__version__)


PyTorch version: 2.5.1+cu124
TorchVision version: 0.20.1+cu124


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os


In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
train_dir = "../Forest Fire Dataset/Training"  # Replace with the path to your dataset

# Data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet
        # transforms.RandomHorizontalFlip(),  # Data augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])
}

# Dataset and DataLoader
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms["train"])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)



In [6]:
# Model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Adjust final layer for binary classification
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Training loop
num_epochs = 20  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradient
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")



Epoch 1/20, Loss: 0.0960, Accuracy: 0.9724
Epoch 2/20, Loss: 0.0784, Accuracy: 0.9776
Epoch 3/20, Loss: 0.0573, Accuracy: 0.9776
Epoch 4/20, Loss: 0.0829, Accuracy: 0.9704
Epoch 5/20, Loss: 0.1014, Accuracy: 0.9698
Epoch 6/20, Loss: 0.0612, Accuracy: 0.9796
Epoch 7/20, Loss: 0.0413, Accuracy: 0.9869
Epoch 8/20, Loss: 0.0536, Accuracy: 0.9783
Epoch 9/20, Loss: 0.0366, Accuracy: 0.9888
Epoch 10/20, Loss: 0.0951, Accuracy: 0.9730
Epoch 11/20, Loss: 0.1023, Accuracy: 0.9678
Epoch 12/20, Loss: 0.0603, Accuracy: 0.9796
Epoch 13/20, Loss: 0.0368, Accuracy: 0.9934
Epoch 14/20, Loss: 0.1655, Accuracy: 0.9494
Epoch 15/20, Loss: 0.1182, Accuracy: 0.9573
Epoch 16/20, Loss: 0.0789, Accuracy: 0.9730
Epoch 17/20, Loss: 0.0791, Accuracy: 0.9744
Epoch 18/20, Loss: 0.0457, Accuracy: 0.9855
Epoch 19/20, Loss: 0.0243, Accuracy: 0.9928
Epoch 20/20, Loss: 0.0929, Accuracy: 0.9619


In [8]:
# Save the trained model
torch.save(model.state_dict(), "deepfire_resnet50.pth")
# print("Model saved as 'deepfire_resnet50.pth'")